In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

1. Objective of this notebook

I tried to find the difference in the features between people who bought online and did not it. And next, I tried to find the suitable model for prediction.

2. Approach

1) Data outlines

2) Visualization

3) Clustering to understand the difference in the features between people who bought online and did not it

4) Fiding the sutable model for prediction by using PyCaret

3. Insight

1) I got 4 clusters from data. The ratio of Revenue '1' is higher in cluster '3' and '1'. THe features of cluster '3' and '1' are that they have higher average in 'Administrative', 'Administrative_Duration', 'Informational', 'Informational_Duration', 'ProductRelated' and 'ProductRelated_Duration' , and lower avarage in 'BounceRates' and 'ExitRates' than cluster '0' and '2'.

2) I tried to have prediction model by lightgbm and catboost. I also tried blend model and ensemble model. According to SHAP analysis, 'Pagevalues' and 'Month' are larger impact to 'Revenue'. It means that how we can improve the pagevalues and when we will promote it are very important.


1) Data outlines

In [ ]:
df=pd.read_csv('/kaggle/input/online-shoppers-purchasing-intention-dataset/online_shoppers_intention.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.nunique()

2) Visualizaton

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df['Month'].unique()

In [ ]:
df=df.replace({'Feb':2, 'Mar':3, 'May':5, 'Oct':10, 'June':6, 'Jul':7, 'Aug':8, 'Nov':11, 'Sep':9,
       'Dec':10})

In [ ]:
df['VisitorType'].unique()

In [ ]:
df=df.replace({'Returning_Visitor':0, 'New_Visitor':1, 'Other':2})

In [ ]:
df=df*1

In [ ]:
df.hist(column='Administrative', by='Revenue', sharex=True, sharey=True, figsize=(7,7), layout=(2,2))
plt.show()

g = sns.FacetGrid(df, hue='Revenue', height=7)
g.map(sns.distplot, 'Administrative', label='Revenue')
g.add_legend()
plt.show()

In [ ]:
df.hist(column='Administrative_Duration', by='Revenue', sharex=True, sharey=True, figsize=(7,7), layout=(2,2))
plt.show()

g = sns.FacetGrid(df, hue='Revenue', height=7)
g.map(sns.distplot, 'Administrative_Duration', label='Revenue')
g.add_legend()
plt.show()

In [ ]:
df.hist(column='Informational', by='Revenue', sharex=True, sharey=True, figsize=(7,7), layout=(2,2))
plt.show()

g = sns.FacetGrid(df, hue='Revenue', height=7)
g.map(sns.distplot, 'Informational', label='Revenue')
g.add_legend()
plt.show()

In [ ]:
df.hist(column='Informational_Duration', by='Revenue', sharex=True, sharey=True, figsize=(7,7), layout=(2,2))
plt.show()

g = sns.FacetGrid(df, hue='Revenue', height=7)
g.map(sns.distplot, 'Informational_Duration', label='Revenue')
g.add_legend()
plt.show()

In [ ]:
df.hist(column='ProductRelated', by='Revenue', sharex=True, sharey=True, figsize=(7,7), layout=(2,2))
plt.show()

g = sns.FacetGrid(df, hue='Revenue', height=7)
g.map(sns.distplot, 'ProductRelated', label='Revenue')
g.add_legend()
plt.show()

In [ ]:
df.hist(column='ProductRelated_Duration', by='Revenue', sharex=True, sharey=True, figsize=(7,7), layout=(2,2))
plt.show()

g = sns.FacetGrid(df, hue='Revenue', height=7)
g.map(sns.distplot, 'ProductRelated_Duration', label='Revenue')
g.add_legend()
plt.show()

In [ ]:
df.hist(column='BounceRates', by='Revenue', sharex=True, sharey=True, figsize=(7,7), layout=(2,2))
plt.show()

g = sns.FacetGrid(df, hue='Revenue', height=7)
g.map(sns.distplot, 'BounceRates', label='Revenue')
g.add_legend()
plt.show()

In [ ]:
df.hist(column='ExitRates', by='Revenue', sharex=True, sharey=True, figsize=(7,7), layout=(2,2))
plt.show()

g = sns.FacetGrid(df, hue='Revenue', height=7)
g.map(sns.distplot, 'ExitRates', label='Revenue')
g.add_legend()
plt.show()

In [ ]:
df.hist(column='PageValues', by='Revenue', sharex=True, sharey=True, figsize=(7,7), layout=(2,2))
plt.show()

g = sns.FacetGrid(df, hue='Revenue', height=7)
g.map(sns.distplot, 'PageValues', label='Revenue')
g.add_legend()
plt.show()

In [ ]:
df.hist(column='SpecialDay', by='Revenue', sharex=True, sharey=True, figsize=(7,7), layout=(2,2))
plt.show()

g = sns.FacetGrid(df, hue='Revenue', height=7)
g.map(sns.distplot, 'SpecialDay', label='Revenue')
g.add_legend()
plt.show()

In [ ]:
df.hist(column='Month', by='Revenue', sharex=True, sharey=True, figsize=(7,7), layout=(2,2))
plt.show()

g = sns.FacetGrid(df, hue='Revenue', height=7)
g.map(sns.distplot, 'Month', label='Revenue')
g.add_legend()
plt.show()

In [ ]:
df.hist(column='OperatingSystems', by='Revenue', sharex=True, sharey=True, figsize=(7,7), layout=(2,2))
plt.show()

g = sns.FacetGrid(df, hue='Revenue', height=7)
g.map(sns.distplot, 'OperatingSystems', label='Revenue')
g.add_legend()
plt.show()

In [ ]:
df.hist(column='Browser', by='Revenue', sharex=True, sharey=True, figsize=(7,7), layout=(2,2))
plt.show()

g = sns.FacetGrid(df, hue='Revenue', height=7)
g.map(sns.distplot, 'Browser', label='Revenue')
g.add_legend()
plt.show()

In [ ]:
df.hist(column='Region', by='Revenue', sharex=True, sharey=True, figsize=(7,7), layout=(2,2))
plt.show()

g = sns.FacetGrid(df, hue='Revenue', height=7)
g.map(sns.distplot, 'Region', label='Revenue')
g.add_legend()
plt.show()

In [ ]:
df.hist(column='TrafficType', by='Revenue', sharex=True, sharey=True, figsize=(7,7), layout=(2,2))
plt.show()

g = sns.FacetGrid(df, hue='Revenue', height=7)
g.map(sns.distplot, 'TrafficType', label='Revenue')
g.add_legend()
plt.show()

In [ ]:
df.hist(column='VisitorType', by='Revenue', sharex=True, sharey=True, figsize=(7,7), layout=(2,2))
plt.show()

g = sns.FacetGrid(df, hue='Revenue', height=7)
g.map(sns.distplot, 'VisitorType', label='Revenue')
g.add_legend()
plt.show()

In [ ]:
df.hist(column='Weekend', by='Revenue', sharex=True, sharey=True, figsize=(7,7), layout=(2,2))
plt.show()

g = sns.FacetGrid(df, hue='Revenue', height=7)
g.map(sns.distplot, 'Weekend', label='Revenue')
g.add_legend()
plt.show()

3) Clustering

In [ ]:
pip install pycaret

In [ ]:
from pycaret.clustering import *

In [ ]:
exp_clust = setup(df, normalize = True,session_id = 123,silent=True)

In [ ]:
kmeans = create_model('kmeans',num_clusters = 4)

In [ ]:
plot_model(kmeans, plot = 'elbow')

In [ ]:
kmean_results = assign_model(kmeans)
kmean_results.groupby(['Cluster','Revenue'])['Revenue'].count().plot.bar()

In [ ]:
pd.set_option('display.max_rows', 150)
kmean_results.groupby('Cluster').describe().T

In [ ]:
plot_model(kmeans)

In [ ]:
plot_model(kmeans, plot = 'silhouette')

4) Modeling for Prediction

In [ ]:
from pycaret.classification import *

In [ ]:
exp_clf = setup(df, target = 'Revenue',silent=True)

In [ ]:
lgm=create_model('lightgbm')

In [ ]:
tuned_lgm=tune_model(lgm)

In [ ]:
plot_model(tuned_lgm)

In [ ]:
plot_model(tuned_lgm,'confusion_matrix')

In [ ]:
interpret_model(tuned_lgm)

In [ ]:
interpret_model(tuned_lgm, plot = 'correlation')

In [ ]:
interpret_model(tuned_lgm, plot = 'reason', observation = 10)

In [ ]:
catboost = create_model('catboost')

In [ ]:
tuned_catboost=tune_model(catboost)

In [ ]:
plot_model(tuned_catboost)

In [ ]:
plot_model(tuned_catboost,'confusion_matrix')

In [ ]:
interpret_model(tuned_catboost)

In [ ]:
interpret_model(tuned_catboost, plot = 'correlation')

In [ ]:
interpret_model(tuned_catboost, plot = 'reason', observation = 10)

In [ ]:
blender = blend_models(estimator_list = [tuned_lgm,tuned_catboost]) 

In [ ]:
plot_model(blender)

In [ ]:
plot_model(blender,'confusion_matrix')

In [ ]:
pred_result = predict_model(blender)

In [ ]:
ensembled_lgm = ensemble_model(tuned_lgm)

In [ ]:
plot_model(ensembled_lgm)

In [ ]:
plot_model(ensembled_lgm,'confusion_matrix')